In [1]:
import findspark
findspark.init()

import pyspark
import sys
import re
import random

from pyspark import SparkConf, SparkContext

sc = SparkContext(appName = "MovieLens")

sc.addPyFile("similaritycos.py")
import similaritycos
# spark-submit similaritycos.py

from similaritycos import cosine_similarity
#  jaccard_similarity, 


def parseVector(line):
    # line.split(r'::')
    return line.split(r'::')

# def parseRatings(line):
#     # line.split("::")moviePairSimilarities.saveAsTextFile("movie-sims")
#     fields= parseVector(line)
#     return int(fields[0]),(int(fields[1]),float(fields[2]))

def parseMovies(line,movieDict):
    fields= parseVector(line)
    movieDict[int(fields[0])] = fields[1]
    return fields[0],(fields[1],fields[2])

def getSampleInteractions(user, film, n):
    if len(film) > n:
        return user, random.sample(film,n)
    else:
        return user, film
    return

def uniqueFilter((userid, ratings)): #movies the uder has rated 
    (item1, value1) = ratings[0]
    (item2, value2) = ratings[1]
    return item1 < item2

def itemitem((item,values)):
    (itemA, valueA) = values[0]
    (itemB, valueB) = values[1]
    return ((itemA, itemB), (valueA, valueB))


numPartitions = 11
movies_data = sc.textFile("moviesset")
# movieDict={}
movies_header = movies_data.take(1)[0]
# movies= movies_data.map(lambda line: re.split(r'::',line)).map(lambda x: (x[0],(x[1],x[2])))
movies= movies_data.filter(lambda line: line!=movies_header).map(lambda line: re.split(r',',line)).map(lambda x: (x[0],(x[1],x[2])))

ratings_data = sc.textFile("ratingsset")
ratings_header = ratings_data.take(1)[0]
# ratings = ratings_data.map(lambda line: re.split(r'::',line)).map(lambda x: (int(x[0]),(int(x[1]),float(x[2])))).filter(lambda x: x[1][1] >= 4).repartition(numPartitions)

ratings = ratings_data.filter(lambda line: line!=ratings_header).map(lambda line: re.split(r',',line)).map(lambda x: (int(x[0]),(int(x[1]),float(x[2])))).repartition(numPartitions)


#FILES FROM HDFS JUPYTER NOTEBOOK

user_ratings_data= ratings.join(ratings)
uniqueJoin = user_ratings_data.filter(uniqueFilter)

moviePairs = uniqueJoin.map(itemitem)
moviePairs.saveAsTextFile("movie-pairs3")

moviePairRatings = moviePairs.groupByKey()

movie_movie_Similarities = moviePairRatings.mapValues(cosine_similarity).cache()
movie_movie_Similarities.sortByKey()
movie_movie_Similarities.saveAsTextFile("movie-sim1")

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.hadoop.hdfs.protocol.DSQuotaExceededException: The DiskSpace quota of /user/ao309 is exceeded: quota = 5368709120 B = 5 GB but diskspace consumed = 5395141741 B = 5.02 GB
	at org.apache.hadoop.hdfs.server.namenode.DirectoryWithQuotaFeature.verifyDiskspaceQuota(DirectoryWithQuotaFeature.java:149)
	at org.apache.hadoop.hdfs.server.namenode.DirectoryWithQuotaFeature.verifyQuota(DirectoryWithQuotaFeature.java:159)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.verifyQuota(FSDirectory.java:2006)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.updateCount(FSDirectory.java:1873)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.updateCount(FSDirectory.java:1848)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.addBlock(FSDirectory.java:441)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.saveAllocatedBlock(FSNamesystem.java:3840)
	at org.apache.hadoop.

In [ ]:
# if __name__=="__main__":
#     if len(sys.argv)< 3:
#         print >> sys.stderr, "Usage: MovieLens ratings movies"
#         exit(-1)
#   #  print "hello"

#     ratings_file = sys.argv[1]
#     movies_file = sys.argv[2]

#     print "ratings {0}\nmovies {1}".format(ratings_file,movies_file)


In [3]:
if (len(sys.argv)> 1):
    scoreThreshold = 0.97
    coOccurenceThreshold = 50
    movieID = int(sys.argv[1])
    filteredResults = movie_movie_Similarities.filter(lambda((pair,sim)): \
        (pair[0] == movieID or pair[1] == movieID) \ 
        and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

ValueError: invalid literal for int() with base 10: '-f'

In [ ]:
results = filteredResults.map(lambda((pair,sim)): (sim,pair)).sortByKey(ascending = False).take(10)